<a href="https://colab.research.google.com/github/sooonsyk/ESAA/blob/main/cluster_mvm_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
DF_2022=pd.read_csv('/content/drive/MyDrive/DF_2022_KNN.csv')
col = ['MESURE_AGE_CO','SEXDSTN_FLAG_CD','MESURE_IEM_001_VALUE','MESURE_IEM_002_VALUE','MESURE_IEM_003_VALUE','MESURE_IEM_005_VALUE','MESURE_IEM_006_VALUE','MESURE_IEM_012_VALUE','MESURE_IEM_018_VALUE']
df_2022 = DF_2022[col]
df_2022 = pd.get_dummies(data=df_2022)

In [3]:
from sklearn.cluster import KMeans

In [4]:
kmeans = KMeans(n_clusters=7, init='k-means++', max_iter=300, random_state=42)
kmeans.fit(df_2022)
df_2022['cluster'] = kmeans.labels_
df_2022['MVM_PRSCRPTN_CN'] = DF_2022['df_2022[, 16]']
df_2022['cluster'].value_counts()

0    31683
2    26071
3    19379
1    19204
4    17492
6        3
5        2
Name: cluster, dtype: int64

In [5]:
df_2022_c0 = df_2022[df_2022['cluster']==0]
df_2022_c1 = df_2022[df_2022['cluster']==1]
df_2022_c2 = df_2022[df_2022['cluster']==2]
df_2022_c3 = df_2022[df_2022['cluster']==3]
df_2022_c4 = df_2022[df_2022['cluster']==4]
df_2022_c5 = df_2022[df_2022['cluster']==5]
df_2022_c6 = df_2022[df_2022['cluster']==6]

In [6]:
#운동리스트 생성 함수
def mvm_list(data):
  data_mvm = data['MVM_PRSCRPTN_CN'].dropna(axis=0)

  data_ispre = data_mvm[data_mvm.str.startswith('준')]  #준비운동 포함
  data_nopre = data_mvm[data_mvm.str.startswith('준')==False] # 준비운동 포함 안 함

  ispre_nomain = data_ispre.str.split('본운동:',expand=True).reset_index(drop=True)[:][0]
  ispre_ismain = data_ispre.str.split('본운동:',expand=True).reset_index(drop=True)[:][1]
  ispre_ismain.dropna(axis=0, inplace=True)

  ispre_nomain_noend = ispre_nomain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][0] 
  ispre_nomain_isend = ispre_nomain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][1]
  ispre_nomain_isend.dropna(axis=0, inplace=True)

  ispre_ismain_noend = ispre_ismain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][0] 
  ispre_ismain_isend = ispre_ismain.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][1] 
  ispre_ismain_isend.dropna(axis=0, inplace=True)

  nopre_noend = data_nopre.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][0]
  nopre_isend = data_nopre.str.split('마무리운동:',expand=True).reset_index(drop=True)[:][1]
  nopre_isend.dropna(axis=0,inplace=True)

  #준비운동 리스트 만들기
  ispre_nomain_noend = ispre_nomain_noend.str.replace('준비운동:','')
  ispre_nomain_noend = ispre_nomain_noend.str.replace(' / ','')
  pre_data = ispre_nomain_noend.str.split(',')
  pre_list = []
  for i in range(pre_data.shape[0]):
    for j in range(len(pre_data[i])):
      pre_list.append(pre_data[i][j])
  pre_list = pd.DataFrame(pre_list)

  #본운동 리스트 만들기
  nopre_noend = nopre_noend.str.replace('본운동:','')
  nopre_noend = nopre_noend.str.replace(' / ','')
  nopre_noend = nopre_noend.str.split(',')
  ispre_ismain_noend = ispre_ismain_noend.str.replace(' / ','')
  ispre_ismain_noend = ispre_ismain_noend.str.split(',')
  main_data = pd.concat([nopre_noend, ispre_ismain_noend],axis=0)
  main_data.reset_index(drop=True, inplace=True)
  main_list = []
  for i in range(main_data.shape[0]):
    for j in range(len(main_data[i])):
        main_list.append(main_data[i][j])
  main_list = pd.DataFrame(main_list)

  #마무리운동 리스트 만들기
  ispre_nomain_isend = ispre_nomain_isend.str.split(',')
  ispre_ismain_isend = ispre_ismain_isend.str.split(',')
  nopre_isend = nopre_isend.str.split(',')
  end_data = pd.concat([ispre_nomain_isend, ispre_ismain_isend, nopre_isend], axis=0)
  end_data.reset_index(drop=True, inplace=True)
  end_list = []
  for i in range(end_data.shape[0]):
    for j in range(len(end_data[i])):
        end_list.append(end_data[i][j])
  end_list = pd.DataFrame(end_list)
  
  return pre_list, main_list, end_list

In [101]:
#클러스터별 운동 리스트 생성
c0_pre, c0_main, c0_end = mvm_list(df_2022_c0)
c1_pre, c1_main, c1_end = mvm_list(df_2022_c1)
c2_pre, c2_main, c2_end = mvm_list(df_2022_c2)
c3_pre, c3_main, c3_end = mvm_list(df_2022_c3)
c4_pre, c4_main, c4_end = mvm_list(df_2022_c4)

In [168]:
#클러스터와 원하는 출력 개수 비율 입력하면 준비, 본, 마무리 운동별 운동처방리스트 생성
def print_mvm(cluster, ratio):
  if cluster == 0:
    pre_list = list((c0_pre[0].value_counts()[c0_pre[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    main_list = list((c0_main[0].value_counts()[c0_main[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    end_list = list((c0_end[0].value_counts()[c0_end[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)

  elif cluster == 1:
    pre_list = list((c1_pre[0].value_counts()[c1_pre[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    main_list = list((c1_main[0].value_counts()[c1_main[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    end_list = list((c1_end[0].value_counts()[c1_end[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)

  elif cluster == 2:
    pre_list = list((c2_pre[0].value_counts()[c2_pre[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    main_list = list((c2_main[0].value_counts()[c2_main[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    end_list = list((c2_end[0].value_counts()[c2_end[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)

  elif cluster == 3:
    pre_list = list((c3_pre[0].value_counts()[c3_pre[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    main_list = list((c3_main[0].value_counts()[c3_main[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    end_list = list((c3_end[0].value_counts()[c3_end[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)

  elif cluster == 4:
    pre_list = list((c4_pre[0].value_counts()[c4_pre[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    main_list = list((c4_main[0].value_counts()[c4_main[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)
    end_list = list((c4_end[0].value_counts()[c4_end[0].value_counts()>df_2022['cluster'].value_counts()[cluster]*ratio]).index)

  return pre_list, main_list, end_list

In [169]:
test1_pre, test1_main, test1_end = print_mvm(3,0.1)

In [171]:
list((c3_pre[0].value_counts()[c3_pre[0].value_counts()>df_2022['cluster'].value_counts()[3]*0.1]).index)

['전신 루틴 스트레칭', '동적 스트레칭 루틴프로그램', '유산소 운동 전 동적 루틴 스트레칭']

In [172]:
test1_pre

['전신 루틴 스트레칭', '동적 스트레칭 루틴프로그램', '유산소 운동 전 동적 루틴 스트레칭']